In [2]:
sc

In [22]:
taxi_aug14 = sc.textFile('taxi_aug_14_1000.csv'   , use_unicode=False).cache()
uber_aug14 = sc.textFile('uber-raw-data-aug14.csv', use_unicode=False).cache()
# list(enumerate(taxi_aug14.first().split(',')))
# list(enumerate(uber_aug14.first().split(',')))

[(0, 'VTS'),
 (1, '2014-08-06 07:24:00'),
 (2, '2014-08-06 07:33:00'),
 (3, '1'),
 (4, '1.76'),
 (5, '-73.956327000000002'),
 (6, '40.767992'),
 (7, '1'),
 (8, ''),
 (9, '-73.977829999999997'),
 (10, '40.761262000000002'),
 (11, 'CSH'),
 (12, '8.5'),
 (13, '0'),
 (14, '0.5'),
 (15, '0'),
 (16, '0'),
 (17, '9')]

In [24]:
def extractTaxi(partId, records):
    '''if partId==0:
        records.next()'''
    import csv
    reader = csv.reader(records)
    for row in reader:
        (pickup, lon, lat) = (row[1], row[5], row[6])
        yield pickup, lon, lat

trdd = taxi_aug14.mapPartitionsWithIndex(extractTaxi)
trdd.take(10)

[('2014-08-06 07:24:00', '-73.956327000000002', '40.767992'),
 ('2014-08-17 03:01:00', '-74.001401999999999', '40.71752'),
 ('2014-08-09 18:50:20', '-73.987528999999995', '40.743077'),
 ('2014-08-06 13:25:17', '-73.977215000000001', '40.758662000000001'),
 ('2014-08-02 00:43:00', '-73.977147000000002', '40.784776999999998'),
 ('2014-08-05 10:33:00', '-73.957759999999993', '40.776201999999998'),
 ('2014-08-02 19:27:12', '-73.978440000000006', '40.763021999999999'),
 ('2014-08-21 18:06:00', '-73.981459999999998', '40.749867000000002'),
 ('2014-08-27 22:57:00', '-73.998692000000005', '40.739874999999998'),
 ('2014-08-17 10:21:00', '-73.782110000000003', '40.644677000000001')]